In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 7)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-07
yesterday: 2023-07-06


#### Restart and Run All

In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1276452,WHART,2023-07-06,2023-07-06,0,10.60,10.50,-0.10,-0.94%,-1,0,10.70,10.50,"5,679,446",2023-07-06,2023-07-06,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,11.00,11.00,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,5pct,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-07-06'
ORDER BY a.name



,name
0,AH
1,AWC
2,BBL
3,BCH
4,BDMS
5,BEM
6,BH
7,CPNREIT
8,DIF
9,EA


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-07-06' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,CPNREIT,2023-07-05,2023-07-06,10.70,10.80,"118,032,312",11.40,10.50,B,SET,0.10,0.93%
1,DIF,2023-07-04,2023-07-06,11.10,10.90,"691,719,920",11.20,10.60,B,SET,-0.20,-1.80%
2,GVREIT,2023-07-04,2023-07-06,8.45,8.30,"709,600",8.50,8.30,B,SET,-0.15,-1.78%
3,JMT,2023-07-06,2023-07-06,36.25,36.50,"172,491,956",39.25,34.00,B,SET50,0.25,0.69%
4,MCS,2023-07-06,2023-07-06,6.70,6.45,"3,742,072",6.70,6.20,B,SET,-0.25,-3.73%
5,SENA,2023-07-04,2023-07-06,3.02,2.98,"15,040,480",3.02,2.90,B,SET,-0.04,-1.32%
6,SINGER,2023-07-05,2023-07-06,9.75,9.20,"75,775,160",9.75,8.95,B,SET100,-0.55,-5.64%
7,STA,2023-07-06,2023-07-06,17.00,17.30,"22,461,208",17.60,16.60,B,SET100,0.30,1.76%
8,SYNEX,2023-07-06,2023-07-06,12.10,12.00,"9,743,608",12.40,11.80,B,SET,-0.10,-0.83%
9,TFFIF,2023-06-29,2023-07-06,7.55,7.60,"110,344,192",7.65,7.55,B,SET,0.05,0.66%


In [11]:
df.shape

(35, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,CPNREIT,2023-07-05,2023-07-06,10.70,10.80,"118,032,312",11.40,10.50,0.93%,B
1,DIF,2023-07-04,2023-07-06,11.10,10.90,"691,719,920",11.20,10.60,-1.80%,B
2,GVREIT,2023-07-04,2023-07-06,8.45,8.30,"709,600",8.50,8.30,-1.78%,B
3,JMT,2023-07-06,2023-07-06,36.25,36.50,"172,491,956",39.25,34.00,0.69%,B
4,MCS,2023-07-06,2023-07-06,6.70,6.45,"3,742,072",6.70,6.20,-3.73%,B
5,SENA,2023-07-04,2023-07-06,3.02,2.98,"15,040,480",3.02,2.90,-1.32%,B
6,SINGER,2023-07-05,2023-07-06,9.75,9.20,"75,775,160",9.75,8.95,-5.64%,B
7,STA,2023-07-06,2023-07-06,17.00,17.30,"22,461,208",17.60,16.60,1.76%,B
8,SYNEX,2023-07-06,2023-07-06,12.10,12.00,"9,743,608",12.40,11.80,-0.83%,B
9,TFFIF,2023-06-29,2023-07-06,7.55,7.60,"110,344,192",7.65,7.55,0.66%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(35, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,CPNREIT,2023-07-05,2023-07-06,10.70,10.80,118032312,11.40,10.50,0.93,B
1,DIF,2023-07-04,2023-07-06,11.10,10.90,691719920,11.20,10.60,-1.80,B
2,GVREIT,2023-07-04,2023-07-06,8.45,8.30,709600,8.50,8.30,-1.78,B
3,JMT,2023-07-06,2023-07-06,36.25,36.50,172491956,39.25,34.00,0.69,B
4,MCS,2023-07-06,2023-07-06,6.70,6.45,3742072,6.70,6.20,-3.73,B
5,SENA,2023-07-04,2023-07-06,3.02,2.98,15040480,3.02,2.90,-1.32,B
6,SINGER,2023-07-05,2023-07-06,9.75,9.20,75775160,9.75,8.95,-5.64,B
7,STA,2023-07-06,2023-07-06,17.00,17.30,22461208,17.60,16.60,1.76,B
8,SYNEX,2023-07-06,2023-07-06,12.10,12.00,9743608,12.40,11.80,-0.83,B
9,TFFIF,2023-06-29,2023-07-06,7.55,7.60,110344192,7.65,7.55,0.66,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-07-07,1.88,-0.01,-0.529101,1.88,1.90,1.86,14471826,27235.68009
1,ADVANC,2023-07-07,214.00,-2.00,-0.925926,215.00,217.00,214.00,3856294,829041.25900
2,AEONTS,2023-07-07,170.00,0.00,0.000000,169.00,172.50,167.00,442582,75482.53700
3,AH,2023-07-07,35.75,0.75,2.142857,34.75,35.75,34.75,1701433,60206.35025
4,AIMIRT,2023-07-07,11.40,-0.10,-0.869565,11.40,11.40,11.30,179200,2042.87000
...,...,...,...,...,...,...,...,...,...,...
182,WHAIR,2023-07-07,7.05,0.00,0.000000,6.95,7.10,6.95,346100,2409.57500
183,WHART,2023-07-07,10.50,0.00,0.000000,10.60,10.60,10.40,1144922,12066.12320
184,WHAUP,2023-07-07,3.58,0.00,0.000000,3.56,3.60,3.56,315688,1132.85096
185,WICE,2023-07-07,8.45,-0.20,-2.312139,8.65,8.65,8.45,532805,4522.14770


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,CPNREIT,2023-07-05,2023-07-06,10.70,10.80,118032312,11.40,10.50,0.93,B,2023-07-07,11.00,0.20,1.851852,10.80,11.00,10.70,3803520,4.117064e+04
1,DIF,2023-07-04,2023-07-06,11.10,10.90,691719920,11.20,10.60,-1.80,B,2023-07-07,10.90,0.00,0.000000,10.80,10.90,10.70,8102882,8.778858e+04
2,GVREIT,2023-07-04,2023-07-06,8.45,8.30,709600,8.50,8.30,-1.78,B,2023-07-07,8.25,-0.05,-0.602410,8.30,8.35,8.25,119500,9.912500e+02
3,JMT,2023-07-06,2023-07-06,36.25,36.50,172491956,39.25,34.00,0.69,B,2023-07-07,36.25,-0.25,-0.684932,36.00,37.25,35.50,11434547,4.161724e+05
4,MCS,2023-07-06,2023-07-06,6.70,6.45,3742072,6.70,6.20,-3.73,B,2023-07-07,6.45,0.00,0.000000,6.45,6.55,6.40,421504,2.729371e+03
5,SENA,2023-07-04,2023-07-06,3.02,2.98,15040480,3.02,2.90,-1.32,B,2023-07-07,2.98,0.00,0.000000,2.98,2.98,2.94,245938,7.300548e+02
6,SINGER,2023-07-05,2023-07-06,9.75,9.20,75775160,9.75,8.95,-5.64,B,2023-07-07,9.15,-0.05,-0.543478,9.15,9.30,9.00,3468892,3.185032e+04
7,STA,2023-07-06,2023-07-06,17.00,17.30,22461208,17.60,16.60,1.76,B,2023-07-07,17.20,-0.10,-0.578035,17.30,17.40,17.00,2219374,3.818143e+04
8,SYNEX,2023-07-06,2023-07-06,12.10,12.00,9743608,12.40,11.80,-0.83,B,2023-07-07,12.00,0.00,0.000000,12.00,12.10,11.90,80971,9.702858e+02
9,TFFIF,2023-06-29,2023-07-06,7.55,7.60,110344192,7.65,7.55,0.66,B,2023-07-07,7.60,0.00,0.000000,7.55,7.60,7.55,1130900,8.589120e+03


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,CPNREIT,2023-07-05,2023-07-06,10.70,10.80,118032312,11.40,10.50,0.93,B,11.00,0.20,3803520,2023-07-07
1,DIF,2023-07-04,2023-07-06,11.10,10.90,691719920,11.20,10.60,-1.80,B,10.90,0.00,8102882,2023-07-07
2,GVREIT,2023-07-04,2023-07-06,8.45,8.30,709600,8.50,8.30,-1.78,B,8.25,-0.05,119500,2023-07-07
3,JMT,2023-07-06,2023-07-06,36.25,36.50,172491956,39.25,34.00,0.69,B,36.25,-0.25,11434547,2023-07-07
4,MCS,2023-07-06,2023-07-06,6.70,6.45,3742072,6.70,6.20,-3.73,B,6.45,0.00,421504,2023-07-07
5,SENA,2023-07-04,2023-07-06,3.02,2.98,15040480,3.02,2.90,-1.32,B,2.98,0.00,245938,2023-07-07
6,SINGER,2023-07-05,2023-07-06,9.75,9.20,75775160,9.75,8.95,-5.64,B,9.15,-0.05,3468892,2023-07-07
7,STA,2023-07-06,2023-07-06,17.00,17.30,22461208,17.60,16.60,1.76,B,17.20,-0.10,2219374,2023-07-07
8,SYNEX,2023-07-06,2023-07-06,12.10,12.00,9743608,12.40,11.80,-0.83,B,12.00,0.00,80971,2023-07-07
9,TFFIF,2023-06-29,2023-07-06,7.55,7.60,110344192,7.65,7.55,0.66,B,7.60,0.00,1130900,2023-07-07


In [20]:
df_merge[colu].shape

(35, 14)

In [21]:
df_merge.shape

(35, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-07-05 2023-07-07  34.50  35.75   3.62%  34,643,124  35.75  30.75 I Update
AWC      2023-07-05 2023-07-07   4.88   4.60  -5.74% 244,093,112   4.98   4.60 I Update
BBL      2023-07-05 2023-07-07 160.50 156.00   -2.8% 183,123,192 160.50 155.00 I Update
BCH      2023-07-04 2023-07-07  17.80  17.40  -2.25% 189,494,568  17.80  16.70 I Update
BDMS     2023-07-06 2023-07-07  27.75  26.75   -3.6% 1,080,573,124  28.50  26.75 O Update
BEM      2023-07-05 2023-07-07   8.60   8.30  -3.49% 194,827,264   8.60   8.30 O Update
BH       2023-06-30 2023-07-07 226.00 220.00  -2.65%  35,935,408 228.00 220.00 O Update
CPNREIT  2023-07-05 2023-07-07  10.70  11.00    2.8% 118,032,312  11.40  10.50 B Update
DIF      2023-07-04 2023-07-07  11.10  10.90   -1.8% 691,719,920  11.20  10.60 B Update
EA       2023-07-03 2023-07-07

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb